## Técnicas de bagging para la clasificación

En este cuaderno vamos a entrenar modelos de bagging para la predicción de una variable categórica, en concreto, la supervivencia de los pasajeros del Titanic. Para ello emplearemos los siguientes módulos:

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

Cargamos los datos de entrenamiento y validación a partir de los csv generados anteriormente:

In [ ]:
X_train = pd.read_csv('./data/xtrain_tit.csv')
X_test = pd.read_csv('./data/xtest_tit.csv')
y_train = pd.read_csv('./data/ytrain_tit.csv')
y_test = pd.read_csv('./data/ytest_tit.csv')

Comprobamos que la carga se ha realizado de manera correcta:

In [ ]:
X_train.head()

In [ ]:
X_train.shape

In [ ]:
X_test.head()

In [ ]:
X_test.shape

## Construcción del modelo

Con los datos ya divididos comenzamos la construcción del modelo. Si recordamos la parte teórica el bagging se basa en la agrupación de varios árboles para la obtención de un único resultado. Por ello comenzamos fijando el árbol a partir del cual generaremos el conjunto árboles. Instanciamos el modelo:

In [ ]:
from sklearn.tree import DecisionTreeClassifier 
base_cls = DecisionTreeClassifier() 

Instanciamos así nuestro modelo de bagging indicándole el estimador base que utilizaremos (un árbol de decisión de clasificación) y cuántos  árboles usaremos:

In [ ]:
from sklearn.ensemble import BaggingClassifier
classifierBag = BaggingClassifier(base_estimator = base_cls, random_state=121)

Una vez instanciado el modelo empleamos los datos de entrenamiento para ajustarlo:

In [ ]:
classifierBag.fit(X_train, y_train)

A partir del modelo ajustado podemos realizar predicciones. Realizamos las predicciones sobre el conjunto de validación:

In [ ]:
y_bag_pred = classifierBag.predict(X_test)

Calculamos la precisión en entrenamiento y test:

In [ ]:
classifierBag.score(X_train, y_train)

In [ ]:
classifierBag.score(X_test, y_test)

Apreciamos un cierto overfitting aunque no es demasiado fuerte. Si probamos otros valores parece reducirse el overfitting aunque los resultados finales se mantienen estables. Esto podría considerarse una buena señal pues al menos nuestro modelo generaliza y no memoriza los datos. Veámoslo:

In [ ]:
num_trees = 500
max_samples = 65

In [ ]:
classifierBag_tuned = BaggingClassifier(base_estimator = base_cls, 
                          n_estimators = num_trees, max_samples=max_samples, random_state=121)

In [ ]:
classifierBag_tuned.fit(X_train, y_train)

In [ ]:
classifierBag_tuned.score(X_train, y_train)

In [ ]:
classifierBag_tuned.score(X_test, y_test)

Calculamos algunas de las métricas más relevantes para este modelo. Comenzamos con la matriz de confusión:

In [ ]:
y_bag_pred_tuned = classifierBag_tuned.predict(X_test)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix = confusion_matrix(y_test, y_bag_pred_tuned)
print(confusion_matrix)

Y calculamos también un compendio de distintas métricas para evaluar la calidad:

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_bag_pred_tuned))

Por último calculamos la curva ROC para evaluar la calidad general del modelo para cualquier umbral:

In [ ]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
logit_roc_auc = roc_auc_score(y_test, classifierBag_tuned.predict(X_test))
fpr, tpr, thresholds = roc_curve(y_test, classifierBag_tuned.predict_proba(X_test)[:,1])
plt.figure()
plt.plot(fpr, tpr, label='Clasificador Bagging (Área bajo la curva = %0.2f)' % logit_roc_auc)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Ratio de falsos positivos')
plt.ylabel('Ratio de verdaderos positivos')
plt.title('Curva ROC')
plt.legend(loc="lower right")
#plt.savefig('Log_ROC') si descomentas esta línea puedes guardar la gráfica
plt.show()

Por último pese a haber mejorado la precisión no hemos perdido toda la explicabilidad ya que podemos ver cuáles han sido las variables más relevantes:

In [ ]:
feature_importances = np.mean([
    tree.feature_importances_ for tree in classifierBag.estimators_
], axis=0)
feature_importances

Para visualizar esta importancia construimos un diccionario:

In [ ]:
i = 0
feature_importance_dic = {}
for element in feature_importances:
    feature_importance_dic[X_train.columns[i]] = element
    i = i + 1
feature_importance_dic

Observamos que las variables más relevantes son la edad y el género seguidas por si las personas van o no en tercera clase.